___

#### DS7333 Quantifying the World: Case Study 1
#### DJ Dawkins, Nathan Deinlein, Joseph Lazarus,Puri Rudick
___


Things to do 

**Assumption Check**<br />
**L1  + L2**<br />
**Feature importance**<br />
**Random Forest feature importance**<br />

___
The goal for this case study is to build a linear regression model using L1 or L2 regularization (or both) the task to predict the Critical Temperature at which the feature, 'Material' Super Conducts. In addition, we will evaluate which variable carries the most importance.
___
<a id="main"></a>
The approach for this Notebook will cover the following topics: <br />
1) **Imports and Data Prep**
<br />
2) <a  href="#EDA">EDA  </a>: check missing values, impute as needed, visualizations and summary statistics, outliers, multicollinearity
<br />
3) <a href="#Prep">Prepare Data  </a> : Set aside 10% of data as validation set. Employ 10-fold CV for training the Liniear Regression model.
<br />
4) <a href="#resids">Residual check  </a> : on regression models
<br />
5) <a href="#LR">Linear Regression </a> ,Using L1, L2 models: compare error metrics and feature importance
<br />
6) **Run tuned models** on validation set and compare metrics to evaluate overfitting  
<br />

<a id="prep"></a>
___
# Data Preparation
___
Cleaning raw data and new feature creation

- Imports
- load data
 - Data Quality using Pandas_Profiling  
> - Check missing values
> - check for duplicates

In [1]:
# Imports
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
import sklearn
pd.set_option("display.max_columns", None, "display.max_rows", None)

In [2]:
#load data from Github
train = pd.read_csv("https://raw.githubusercontent.com/JosephLazarus/QTW/main/Case%20Study%201/Data/train.csv")
train.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,53.622535,775.425,1010.268571,718.152900,938.016780,1.305967,0.791488,810.6,735.985714,323.811808,355.562967,160.25,105.514286,136.126003,84.528423,1.259244,1.207040,205,42.914286,75.237540,69.235569,4654.35725,2961.502286,724.953211,53.543811,1.033129,0.814598,8958.571,1579.583429,3306.162897,3572.596624,81.8375,111.727143,60.123179,99.414682,1.159687,0.787382,127.05,80.987143,51.433712,42.558396,6.9055,3.846857,3.479475,1.040986,1.088575,0.994998,12.878,1.744571,4.599064,4.666920,107.756645,61.015189,7.062488,0.621979,0.308148,0.262848,399.97342,57.127669,168.854244,138.517163,2.25,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.0
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,53.979870,766.440,1010.612857,720.605511,938.745413,1.544145,0.807078,810.6,743.164286,290.183029,354.963511,161.20,104.971429,141.465215,84.370167,1.508328,1.204115,205,50.571429,67.321319,68.008817,5821.48580,3021.016571,1237.095080,54.095718,1.314442,0.914802,10488.571,1667.383429,3767.403176,3632.649185,90.8900,112.316429,69.833315,101.166398,1.427997,0.838666,127.05,81.207857,49.438167,41.667621,7.7844,3.796857,4.403790,1.035251,1.374977,1.073094,12.878,1.595714,4.473363,4.603000,172.205316,61.372331,16.064228,0.619735,0.847404,0.567706,429.97342,51.413383,198.554600,139.630922,2.00,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.0
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,53.656268,775.425,1010.820000,718.152900,939.009036,1.305967,0.773620,810.6,743.164286,323.811808,354.804183,160.25,104.685714,136.126003,84.214573,1.259244,1.132547,205,49.314286,75.237540,67.797712,4654.35725,2999.159429,724.953211,53.974022,1.033129,0.760305,8958.571,1667.383429,3306.162897,3592.019281,81.8375,112.213571,60.123179,101.082152,1.159687,0.786007,127.05,81.207857,51.433712,41.639878,6.9055,3.822571,3.479475,1.037439,1.088575,0.927479,12.878,1.757143,4.599064,4.649635,107.756645,60.943760,7.062488,0.619095,0.308148,0.250477,399.97342,57.127669,168.854244,138.540613,2.25,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.0
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,53.639405,775.425,1

In [3]:
# Load data set called  unique_m from github 
unique_m=pd.read_csv("https://raw.githubusercontent.com/JosephLazarus/QTW/main/Case%20Study%201/Data/unique_m.csv")

unique_m.head()

,H,He,Li,Be,B,C,N,O,F,Ne,Na,Mg,Al,Si,P,S,Cl,Ar,K,Ca,Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn,Ga,Ge,As,Se,Br,Kr,Rb,Sr,Y,Zr,Nb,Mo,Tc,Ru,Rh,Pd,Ag,Cd,In,Sn,Sb,Te,I,Xe,Cs,Ba,La,Ce,Pr,Nd,Pm,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,Ta,W,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp,material
0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.20,1.80,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,29.0,Ba0.2La1.8Cu1O4
1,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.10,1.90,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,26.0,Ba0.1La1.9Ag0.1Cu0.9O4
2,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.10,1.90,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,19.0,Ba0.1La1.9Cu1O4
3,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.15,1.85,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,22.0,Ba0.15La1.85Cu1O4
4,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.30,1.70,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,23.0,Ba0.3La1.7Cu1O4


In [4]:
#Join the two datasets

#delete duplicated column
del train['critical_temp']

#create list of column names
train_cols = train.columns.values.tolist()
unique_cols = unique_m.columns.values.tolist() 
col_list = train_cols + unique_cols

df = pd.concat([train, unique_m], axis = 1, ignore_index=True)
df.columns = col_list

In [5]:
#check shape
df.shape

(21263, 169)

In [6]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [7]:
#only needed to run this cell once

#df.to_csv('full_df.csv', index = None, header=True)

___

#### IDK why but profiler isn't working for me. I'll have to do things the old fashion way
___

In [8]:
# profile = ProfileReport(df, title="Pandas Profiling Report")

In [9]:
# profile

___
#### Old fashioned EDA <a id="EDA"></a>
___

In [10]:
df.describe()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,H,He,Li,Be,B,C,N,O,F,Ne,Na,Mg,Al,Si,P,S,Cl,Ar,K,Ca,Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn,Ga,Ge,As,Se,Br,Kr,Rb,Sr,Y,Zr,Nb,Mo,Tc,Ru,Rh,Pd,Ag,Cd,In,Sn,Sb,Te,I,Xe,Cs,Ba,La,Ce,Pr,Nd,Pm,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,Ta,W,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp
count,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.0,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.0,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.0,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.0,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.0,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.0,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.0,21263.0,21263.0,21263.000000
mean,4.11522

In [11]:
# What is the mean Critical Temp 

df["critical_temp"].mean()

34.42121913535249

## Columns with all zeros:
While there were no missing values observed, the summary stats show that some columns have all zeros and are essentially blank and could be removed

In [12]:
# some of these columns (9) of them are just zeros. Removing these columns

# IDK why this line doesn't work for me either

#  df.loc[:, (df != 0).any(axis=0)]

In [13]:
max = df.max().to_frame()
max.rename(columns={ max.columns[0]: "Max" }, inplace = True)

min = df.min().to_frame()
min.rename(columns={ min.columns[0]: "Min" }, inplace = True)

minmax = pd.concat([min,max], axis=1)
#minmax.head(100)

minmax0 = minmax[(minmax['Min'] == 0) & (minmax['Max'] == 0)]
#minmax0.head(100)

all_zeros = minmax0.index.tolist()
print(all_zeros)


['He', 'Ne', 'Ar', 'Kr', 'Xe', 'Pm', 'Po', 'At', 'Rn']


## Remove empty columns

In [14]:
print("Before dropping: ",df.shape)
df = df.drop(all_zeros, axis=1)
print("After dropping: ",df.shape)


Before dropping:  (21263, 169)
After dropping:  (21263, 160)


## Check for Duplicate Data

In [15]:
# Check for duplicate rows
print('Duplicate rows: ',df.duplicated(keep='first').sum())


Duplicate rows:  0


There are not exact duplicate rows, but below shows that there are duplicate entries for materials


In [16]:
materials_count = pd.DataFrame(df.groupby('material').size())
materials_count.rename(columns={ materials_count.columns[0]: "Count" }, inplace = True)


materials_count.sort_values('Count', ascending = False).head(20)

,Count
material,
Y1Ba2Cu3O7,110
Bi2Sr2Ca1Cu2O8,80
La1.85Sr0.15Cu1O4,51
B2Mg1,46
Tl2Ba2Cu1O6,45
Bi2Sr2Ca1Cu2O,44
Y1Ba2Cu3O,44
Y1Ba2Cu4O8,40
Nd1.85Ce0.15Cu1O4,32


In [17]:
abs(df.corr()['critical_temp']).sort_values(ascending=False)

critical_temp                      1.000000
wtd_std_ThermalConductivity        0.721271
range_ThermalConductivity          0.687654
range_atomic_radius                0.653759
std_ThermalConductivity            0.653632
wtd_mean_Valence                   0.632401
wtd_entropy_atomic_mass            0.626930
wtd_gmean_Valence                  0.615653
wtd_entropy_atomic_radius          0.603494
number_of_elements                 0.601069
range_fie                          0.600790
mean_Valence                       0.600085
wtd_std_atomic_radius              0.599199
entropy_Valence                    0.598591
wtd_entropy_Valence                0.589664
wtd_std_fie                        0.582013
gmean_Valence                      0.573068
entropy_fie                        0.567817
O                                  0.566852
wtd_entropy_FusionHeat             0.563244
std_atomic_radius                  0.559629
entropy_atomic_radius              0.558937
Ba                              

In [18]:
# visualise area-price relationship
sns.regplot(x="wtd_std_ThermalConductivity", y="critical_temp", data=df, fit_reg=False)

<AxesSubplot:xlabel='wtd_std_ThermalConductivity', ylabel='critical_temp'>

In [19]:
sns.heatmap(df.corr(),annot=True,cmap='viridis')

<AxesSubplot:>

___
#### Distribution of Critical Temp

Normal distrbution of residuals is needed for Linear Regression not that critical temperature needs to come from a normal dsitrubtion. Because we can envoke the CLT in our case given the number of observations
___
Looking at those correlations plots got me thinking about doing an assumption check. 

I wonder if the response is normally distrubuted.

In [20]:
import scipy.stats as stats
fig, ax = plt.subplots(1,2, figsize=(24,8))

sns.histplot(df['critical_temp'], ax=ax[0]).set(title='Histogram of Critical Temp')
stats.probplot(df['critical_temp'], dist="norm", plot=ax[1])


((array([-3.99316766, -3.77774718, -3.65990182, ...,  3.65990182,
          3.77774718,  3.99316766]),
  array([2.10e-04, 3.25e-04, 4.00e-04, ..., 1.43e+02, 1.43e+02, 1.85e+02])),
 (31.63755377650955, 34.4212191353525, 0.9234614097220072))

In [21]:
ln_temp = np.log(df['critical_temp'])

fig, ax = plt.subplots(1,2, figsize=(12,4))

sns.histplot(ln_temp, ax=ax[0]).set(title='Logrithm Critical Temp)')
stats.probplot(ln_temp, dist="norm", plot=ax[1])

((array([-3.99316766, -3.77774718, -3.65990182, ...,  3.65990182,
          3.77774718,  3.99316766]),
  array([-8.46840303, -8.03168538, -7.82404601, ...,  4.96284463,
          4.96284463,  5.22035583])),
 (1.5158110844454755, 2.7360290954307422, 0.9610850065987526))

In [22]:
#Split Modeling and Validation Datasets
from sklearn.model_selection import train_test_split
#removing material from dataset
df.drop(columns = ['material'],axis = 1, inplace =True)

train, val = train_test_split(df, test_size=0.1, shuffle=True, random_state=42)
print("Size of modeling set (to be CV split): ",train.shape)
print("Size of hold back validation set: ",val.shape)

Size of modeling set (to be CV split):  (19136, 159)
Size of hold back validation set:  (2127, 159)


In [23]:
# Scale testing data
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
names = train.columns
d = scaler.fit_transform(train)
scaled_train = pd.DataFrame(d, columns=names)
scaled_train.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,H,Li,Be,B,C,N,O,F,Na,Mg,Al,Si,P,S,Cl,K,Ca,Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn,Ga,Ge,As,Se,Br,Rb,Sr,Y,Zr,Nb,Mo,Tc,Ru,Rh,Pd,Ag,Cd,In,Sn,Sb,Te,I,Cs,Ba,La,Ce,Pr,Nd,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,Ta,W,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi,critical_temp
0,0.375,0.344372,0.250917,0.265097,0.164014,0.603526,0.477641,0.590973,0.178875,0.438424,0.524667,0.438940,0.653937,0.382593,0.593090,0.608500,0.379682,0.591798,0.593746,0.629425,0.738600,0.415000,0.224663,0.333209,0.144109,0.595377,0.584797,0.667969,0.209821,0.567789,0.687619,0.196247,0.132376,0.029796,0.002352,0.509637,0.434776,0.396598,0.074715,0.317301,0.345186,0.240627,0.340873,0.144724,0.303868,0.620384,0.465389,0.389599,0.371952,0.333792,0.256760,0.063787,0.034341,0.031089,0.007779,0.535082,0.526789,0.122907,0.017274,0.089069,0.089935,0.336809,0.150218,0.026154,0.001589,0.247665,0.165383,0.930227,0.142307,0.775596,0.649265,0.208333,0.212143,0.202227,0.205662,0.639097,0.525656,0.166667,0.159770,0.144338,0.148476,0.0,0.0,0.0,0.0,0.0,0.0,0.060606,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.010204,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.005389,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.01949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.150810
1,0.125,0.643370,0.818142,0.610516,0.808223,0.322886,0.107263,0.423332,0.746936,0.435764,0.261458,0.358735,0.385541,0.356989,0.393186,0.320167,0.146615,0.073975,0.493452,0.096563,0.060418,0.604000,0.607200,0.603990,0.607196,0.323598,0.169717,0.007812,0.667043,0.008658,0.006177,0.558184,0.701265,0.528758,0.689682,0.327949,0.123519,0.357703,0.629754,0.376712,0.232832,0.180376,0.112477,0.159758,0.105939,0.330289,0.322215,0.157880,0.093691,0.169127,0.101476,0.297562,0.332684,0.294329,0.331473,0.335418,0.154650,0.087805,0.290275,0.089087,0.053405,0.166851,0.139270,0.174459,0.150705,0.425021,0.194975,0.006977,0.114338,0.006977,0.004219,0.666667,0.666667,0.666667,0.666667,0.323604,0.166732,0.000000,0.572066,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016364,0.0,

In [24]:
#Scale validations
scaler = preprocessing.MinMaxScaler()
names = train.columns
d = scaler.fit_transform(val)
scaled_val = pd.DataFrame(d, columns=names)
scaled_val.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,wtd_std_atomic_mass,mean_fie,wtd_mean_fie,gmean_fie,wtd_gmean_fie,entropy_fie,wtd_entropy_fie,range_fie,wtd_range_fie,std_fie,wtd_std_fie,mean_atomic_radius,wtd_mean_atomic_radius,gmean_atomic_radius,wtd_gmean_atomic_radius,entropy_atomic_radius,wtd_entropy_atomic_radius,range_atomic_radius,wtd_range_atomic_radius,std_atomic_radius,wtd_std_atomic_radius,mean_Density,wtd_mean_Density,gmean_Density,wtd_gmean_Density,entropy_Density,wtd_entropy_Density,range_Density,wtd_range_Density,std_Density,wtd_std_Density,mean_ElectronAffinity,wtd_mean_ElectronAffinity,gmean_ElectronAffinity,wtd_gmean_ElectronAffinity,entropy_ElectronAffinity,wtd_entropy_ElectronAffinity,range_ElectronAffinity,wtd_range_ElectronAffinity,std_ElectronAffinity,wtd_std_ElectronAffinity,mean_FusionHeat,wtd_mean_FusionHeat,gmean_FusionHeat,wtd_gmean_FusionHeat,entropy_FusionHeat,wtd_entropy_FusionHeat,range_FusionHeat,wtd_range_FusionHeat,std_FusionHeat,wtd_std_FusionHeat,mean_ThermalConductivity,wtd_mean_ThermalConductivity,gmean_ThermalConductivity,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,range_ThermalConductivity,wtd_range_ThermalConductivity,std_ThermalConductivity,wtd_std_ThermalConductivity,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,H,Li,Be,B,C,N,O,F,Na,Mg,Al,Si,P,S,Cl,K,Ca,Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn,Ga,Ge,As,Se,Br,Rb,Sr,Y,Zr,Nb,Mo,Tc,Ru,Rh,Pd,Ag,Cd,In,Sn,Sb,Te,I,Cs,Ba,La,Ce,Pr,Nd,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,Ta,W,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi,critical_temp
0,0.142857,0.367631,0.391393,0.375155,0.398578,0.364936,0.273996,0.102695,0.254017,0.104707,0.097688,0.278411,0.235067,0.300745,0.238240,0.339674,0.321280,0.076417,0.290994,0.106019,0.100259,0.508065,0.606183,0.485599,0.599787,0.331263,0.253356,0.292,0.600481,0.316017,0.351966,0.277593,0.332552,0.298414,0.336788,0.368155,0.279251,0.143745,0.229081,0.154545,0.139645,0.496610,0.366140,0.491797,0.387004,0.385123,0.373515,0.094269,0.165005,0.106947,0.084259,0.274626,0.262649,0.273602,0.263016,0.362339,0.341214,0.047720,0.118335,0.048586,0.042175,0.179229,0.150930,0.185463,0.180768,0.423361,0.416383,0.013954,0.070906,0.014388,0.012461,0.60,0.581860,0.603554,0.602769,0.335637,0.267354,0.166667,0.393369,0.190500,0.199155,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.055556,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047513
1,0.428571,0.335722,0.359172,0.262095,0.303488,0.638476,0.595869,0.614496,0.182175,0.452606,0.451866,0.387753,0.299364,0.346310,0.258875,0.646553,0.699042,0.642568,0.218809,0.685698,0.613592,0.524194,0.562212,0.387100,0.478235,0.623774,0.639959,0.820,0.335084,0.671219,0.756170,0.152297,0.230083,0.028590,0.078720,0.561655,0.494149,0.396598,0.172585,0.304135,0.325373,0.366768,0.307041,0.252734,0.227703,0.625378,0.571994,0.364040,0.217864,0.356474,0.311400,0.072255,0.085476,0.032537,0.055003,0.584416,0.558132,0.122907,0.054371,0.096165,0.084925,0.342302,0.487495,0.024402,0.086622,0.205794,0.144115,0.930228,0.476670,0.807199,0.918348,0.15,0.097198,0.157731,0.146247,0.668831,0.680278,0.166667,0.081739,0.164978,0.160942,0.0,0.0,0.0,0.0,0.0,0.0,0.029197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.2,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.678043
2,0.571429,0.398757,0.205827,0.316

In [25]:
#Create X and Y for modeling
X_scaled = scaled_train.drop(['critical_temp'],axis=1) #droping target
y_scaled = scaled_train['critical_temp']

In [26]:
Xval_scaled = scaled_val.drop(['critical_temp'],axis=1) #droping target
yval_scaled = scaled_val['critical_temp']

In [27]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

## **Cross Validation**
Cross validation is performed using repeated holdout via KFold()

- 3 folds are used
- A random seed is set so the same random test and training splits are used each time cross validation is performed.

In [28]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

cv_method = KFold(n_splits=5, shuffle=False)


> 
> ## Modeling and Evaluation:

Three regression models were conducted using GridSearchCV to find the best regression model that fit the modeling dataset.
1) Linear Regression
2) LASSO Regression
3) Ridge Regression




In [29]:
#Create function for regression models
from sklearn.model_selection import cross_validate

def EvaluateRegressionEstimator(modelNme, regEstimator, X, y, x_valid, y_valid, cv):
    
    print('Parameterization of the best estimator:',regEstimator,'\n')

    scores = cross_validate(regEstimator, X, y, scoring=errorScoring, cv=cv, return_train_score=True)

    yhat = regEstimator.predict(x_valid)

    #cross val score sign-flips the outputs of MAE
    # https://github.com/scikit-learn/scikit-learn/issues/2439
    scores['test_MAE'] = scores['test_MAE'] * -1
    scores['test_MAPE'] = scores['test_MAPE'] * -1
    scores['test_RMSE'] = scores['test_RMSE'] * -1

    #print mean MAE for all folds 
    maeAvg = scores['test_MAE'].mean()
    print_str = "The average MAE for all cv folds is: \t\t\t {maeAvg:.5}"
    print(print_str.format(maeAvg=maeAvg))

    #print mean test_MAPE for all folds
    mape_avg = scores['test_MAPE'].mean()
    print_str = "The average MAE percentage (MAPE) for all cv folds is: \t {mape_avg:.5}"
    print(print_str.format(mape_avg=mape_avg))

    #print mean RMSE for all folds 
    RMSEavg = scores['test_RMSE'].mean()
    print_str = "The average RMSE for all cv folds is: \t\t\t {RMSEavg:.5}"
    print(print_str.format(RMSEavg=RMSEavg))

    #print R2 for all folds 
    R2avg = scores['test_R2'].mean()
    print_str = "The average R2 for all cv folds is: \t\t\t {R2avg:.5}"
    print(print_str.format(R2avg=R2avg))
    print('---------------------------------------------------------------------')

    model_stat = pd.DataFrame()
    model_stat['Model_name'] = [str(modelNme)]
    model_stat['MAE_model'] = scores['test_MAE'].mean()
    model_stat['MAPE_model'] = scores['test_MAPE'].mean()
    model_stat['RMSE_model'] = scores['test_RMSE'].mean()
    model_stat['R2_model'] = scores['test_R2'].mean()
    model_stat['MAE_validation'] = mean_absolute_error(y_valid, yhat)
    model_stat['MAPE_validation'] = mape(y_valid, yhat)
    model_stat['RMSE_validation'] = rmse(y_valid, yhat)
    model_stat['R2_validation'] = r2_score(y_valid, yhat)

    print('Cross Validation Fold Mean Error Scores')
    scoresResults = pd.DataFrame()
    scoresResults['MAE'] = scores['test_MAE']
    scoresResults['MAPE'] = scores['test_MAPE']
    scoresResults['RMSE'] = scores['test_RMSE']
    scoresResults['R2'] = scores['test_R2']
    print(scoresResults)
    
    return model_stat, model_stat['MAE_validation']

In [30]:


def residualPlot(model, x_train, y_train, x_test, y_test):
    print('\n*********************************************\nPrediction Error, Residual and Cook\'s D Plots\n*********************************************')

    predictions = model.predict(x_train)
    residuals = y_train - predictions
    model_norm_residuals_abs_sqrt = np.sqrt(np.abs(residuals))

    plt.subplots(1, 2, figsize=(14, 4))
    plt.subplot(121)
    visualizer = PredictionError(model)
    visualizer.fit(x_train, y_train)
    visualizer.score(x_test, y_test)
    visualizer.finalize()

    plt.subplot(122)
    visualizer = ResidualsPlot(model, hist=False, qqplot=True)
    visualizer.fit(x_train, y_train)
    visualizer.score(x_test, y_test)
    visualizer.show()
    plt.show()

    plt.subplots(1, 2, figsize=(13,4))    
    plt.subplot(121)
    sns.regplot(predictions, model_norm_residuals_abs_sqrt,
                scatter=True,
                lowess=True,
                line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    plt.ylabel("Standarized residuals")
    plt.xlabel("Fitted value")

    y = np.array(y_train, dtype=float)
    x = np.array(x_train, dtype=float)
    plt.subplot(122)
    visualizer = CooksDistance()
    visualizer.fit(x, y)
    visualizer.show()

In [31]:
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error, r2_score

def rmse(y_actual, y_predicted):
    return np.sqrt(mean_squared_error(y_actual, y_predicted))

def mape(y_actual, y_predicted): 
    mask = y_actual != 0
    return (np.fabs(y_actual - y_predicted)/y_actual)[mask].mean() * 100

#Create scorers for rmse and mape functions
mae_scorer = make_scorer(score_func=mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(score_func=rmse, greater_is_better=False)
mape_scorer = make_scorer(score_func=mape, greater_is_better=False)
r2_scorer = make_scorer(score_func=r2_score, greater_is_better=True)

#Make scorer array to pass into cross_validate() function for producing mutiple scores for each cv fold.
errorScoring = {'MAE':  mae_scorer, 
                'RMSE': rmse_scorer,
                'MAPE': mape_scorer,
                'R2': r2_scorer
               } 

### Create Dataframe to collect all regression model scores

In [32]:
#Create dataframe to collect regression models stat
regModel_stat = pd.DataFrame()

### OLS Regression

In [33]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [34]:
#Create a Linear Regression object and perform a grid search to find the best parameters
from sklearn.linear_model import LinearRegression
from sklearn import metrics 
import math

linreg = LinearRegression()
parameters = {'fit_intercept':(True,False)}

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=linreg
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv_method # KFolds = 10
                   , scoring='neg_mean_squared_error')

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_scaled, y_scaled)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LinearRegression(),
             param_grid={'fit_intercept': (True, False)},
             scoring='neg_mean_squared_error', verbose=1)

### Run tuned model on the train dataset and check performance of the model on validation set


In [35]:
#Create CappedLinearRegression predictions between 0 and 100% using the best parameters for our Linear Regression object
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
l, linMae = EvaluateRegressionEstimator('Linear Regression', regEstimator,  X_scaled, y_scaled, Xval_scaled, yval_scaled, cv_method)
regModel_stat = regModel_stat.append(l)

#Plot rediction Error, Residual and Cook's D
residualPlot(regEstimator, X_scaled, y_scaled, Xval_scaled, yval_scaled)

Parameterization of the best estimator: LinearRegression(fit_intercept=False) 

The average MAE for all cv folds is: 			 0.068737
The average MAE percentage (MAPE) for all cv folds is: 	 1875.3
The average RMSE for all cv folds is: 			 0.13403
The average R2 for all cv folds is: 			 0.37361
---------------------------------------------------------------------
Cross Validation Fold Mean Error Scores
        MAE         MAPE      RMSE        R2
0  0.068114  1748.088404  0.092094  0.752293
1  0.066738  2215.986983  0.089590  0.766375
2  0.070515  2475.690505  0.230517 -0.626040
3  0.070409  2596.374740  0.167162  0.210197
4  0.067908   340.442865  0.090778  0.765219


C:\Users\Joey\AppData\Local\Temp\ipykernel_19004\481230193.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  regModel_stat = regModel_stat.append(l)



*********************************************
Prediction Error, Residual and Cook's D Plots
*********************************************


NameError: name 'PredictionError' is not defined

### Hyperparameter Tuning Using Grid Search Cross-Validation
A common use of cross-validation is for tuning hyperparameters of a model. The most common technique is what is called grid search cross-validation.

In [36]:
# number of features in X_train
len(X_scaled.columns)

158

In [39]:
# step-1: create a cross-validation scheme
# Shuffle or to not. 
# according to documentation : Shuffle = TRUE shuffles the data before splitting into batches. 
#Note that the samples within each split will not be shuffled.
folds = KFold(n_splits = 5, shuffle = False)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'n_features_to_select': list(range(0, 158))}]


# step-3: perform grid search
# 3.1 specify model  rfe = recursive feature elimination
lm = LinearRegression()
lm.fit(X_scaled, y_scaled)
rfe = RFE(lm)             

# 3.2 call GridSearchCV() rfe = recursive feature elimination
model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'neg_mean_squared_error', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      





### To reduce runtime of notebook I # commented out model_cv.fit

In [ ]:
# fit the model
model_cv.fit(X_scaled, y_scaled)  

print(f'Coefficients: {model_cv.coef_}')
print(f'Intercept: {model_cv.intercept_}')
print(f'R^2 score: {model_cv.score(X_scaled, y_scaled)}')

Fitting 5 folds for each of 158 candidates, totalling 790 fits


In [40]:
# cv results
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [ ]:
# plotting cv results
plt.figure(figsize=(16,6))

plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('neg_mean_squared_error')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')

In [ ]:
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)
def linearity_test(model, y):
    '''
    Function for visually inspecting the assumption of linearity in a linear regression model.
    It plots observed vs. predicted values and residuals vs. predicted values.
    
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    '''
    fitted_vals = model.predict()
    resids = model.resid

    fig, ax = plt.subplots(1,2)
    
    sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Observed vs. Predicted Values', fontsize=16)
    ax[0].set(xlabel='Predicted', ylabel='Observed')

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Residuals vs. Predicted Values', fontsize=16)
    ax[1].set(xlabel='Predicted', ylabel='Residuals')

In [ ]:
linearity_test(lm, y_scaled)

### L1 Lasso

<a id="LR"></a>

In [ ]:
#Create a regression object and perform a grid search to find the best parameters
from sklearn.linear_model import Lasso

LassoReg = Lasso(fit_intercept=True, copy_X=True
                ,max_iter=1000, tol=0.0001, random_state=0)

#Test parameters 
LassoAlpha = [0.001, 0.01, 0.1, 0.5, 1]
LassoParameters = {'alpha': LassoAlpha}

#Create a grid search object using the parameters above
LassoGridSearch = GridSearchCV(estimator=LassoReg
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=LassoParameters
                   , cv=cv_method # KFolds = 10
                   , scoring='neg_mean_absolute_error')

#Perform hyperparameter search to find the best combination of parameters for our data
LassoGridSearch.fit(X_scaled, y_scaled)

In [ ]:
print("Best Parameters from search: ",LassoGridSearch.best_estimator_.get_params())

In [ ]:
plt.plot(LassoAlpha, LassoGridSearch.cv_results_['mean_test_score'])
plt.xlabel('Value of alpha penalty term')
plt.ylabel('MAE')

In [ ]:
#Create CappedLinearRegression predictions between 0 and 100% using the best parameters for our Linear Regression object
LassoEstimator = LassoGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
L1, LassoMae = EvaluateRegressionEstimator('LASSO Regression:', LassoEstimator, X_scaled, y_scaled, Xval_scaled, yval_scaled, cv_method)
regModel_stat = regModel_stat.append(L1)

#Plot rediction Error, Residual and Cook's D
residualPlot(LassoEstimator, X_scaled, y_scaled, Xval_scaled, yval_scaled)

In [ ]:
L1_weights = pd.DataFrame(X_scaled.columns)
L1_weights['weight'] = LassoEstimator.coef_

L1_weights.sort_values('weight', ascending = False).head(200)

## L2 Ridge Regression

In [ ]:
#Create a regression object and perform a grid search to find the best parameters
from sklearn.linear_model import Ridge

ridgeReg = Ridge(fit_intercept=True, copy_X=True
                ,max_iter=1000, tol=0.0001, random_state=0)

#Test parameters 
ridgeAlpha = [0.001, 0.01, 0.1, 0.5, 1, 5,10, 20, 50, 100, 200, 300]
ridgeParameters = {'alpha': ridgeAlpha}

#Create a grid search object using the parameters above
ridgeGridSearch = GridSearchCV(estimator=ridgeReg
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=ridgeParameters
                   , cv=cv_method # KFolds = 5
                   , scoring='neg_mean_absolute_error')

#Perform hyperparameter search to find the best combination of parameters for our data
ridgeGridSearch.fit(X_scaled, y_scaled)

In [ ]:
#Create CappedLinearRegression predictions between 0 and 100% using the best parameters for our Linear Regression object
ridgeEstimator = ridgeGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
L2, ridgeMae = EvaluateRegressionEstimator('Ridge Regression:', ridgeEstimator, X_scaled, y_scaled, Xval_scaled, yval_scaled, cv_method)
regModel_stat = regModel_stat.append(L2)

#Plot rediction Error, Residual and Cook's D
residualPlot(ridgeEstimator, X_scaled, y_scaled, Xval_scaled, yval_scaled)

In [ ]:

L2_weights = pd.DataFrame(X_scaled.columns)
L2_weights['weight'] = ridgeEstimator.coef_

L2_weights.sort_values('weight', ascending = False).head(50)

**Ridge Yellow Brick Plot**
<a id="RidgeYB">

In [ ]:
regModel_stat

In [ ]:
sns.scatterplot(data=modeling_df, x="wtd_range_ThermalConductivity", y="critical_temp", size=1, alpha=0.3)

In [ ]:
sns.scatterplot(data=modeling_df, x="mean_ThermalConductivity", y="critical_temp", size=1, alpha=0.3)

In [ ]:
sns.scatterplot(data=modeling_df, x="Ba", y="critical_temp", size=1, alpha=0.3)

In [ ]:
mean_ThermalConductivity = pd.DataFrame(modeling_df.groupby('mean_ThermalConductivity').mean())

In [ ]:
mean_ThermalConductivity.sort_values('mean_ThermalConductivity', ascending = False).head(10)

In [ ]:
sns.scatterplot(data=mean_ThermalConductivity, x="mean_ThermalConductivity", y="critical_temp", size=1, alpha=0.3)